### On its own (MLP1L) (Verification)

In [6]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO


# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = torch.jit.load(r'Models and Data splits\MainModel_MLP1L.pt')
model.to(device)
model.eval()

# Root directory for your adversarial images
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")

# Gather all PNG files recursively
all_image_paths = list(adversarial_root.rglob("*.png"))

# Dictionary to accumulate results by folder
results_by_folder = {}

for file_path in all_image_paths:
    try:
        # Read as grayscale
        image = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # Normalize and flatten
        image = image.astype(np.float32) / 255.0
        vector = image.flatten().reshape(1, -1)
        input_tensor = torch.tensor(vector, dtype=torch.float32, device=device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_tensor)
        predicted = torch.argmax(outputs, dim=1).item()

        # Determine context folder name (e.g. 'Adversarial' subfolder)
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: '495_t9_p2_m814.39'
        fname = file_path.stem
        parts = fname.split("_")           # ['495', 't9', 'p2', 'm814.39'] :contentReference[oaicite:0]{index=0}

        index      = int(parts[0])
        true_label = int(parts[1].lstrip("t"))  # strip leading 't' :contentReference[oaicite:1]{index=1}
        pred_label = int(parts[2].lstrip("p"))  # strip leading 'p'
        magnitude  = float(parts[3].lstrip("m"))  # strip leading 'm'

        # Append to results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":     index,
            "True":      true_label,
            "Adversarial": pred_label,
            "Magnitude": magnitude,
            "ModelPred": predicted
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Prepare and print/copy output tables
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

# Copy everything to the clipboard
pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")

# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")






=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          7
   103     2            8    1133.78          8
   104     2            7     852.70          7
   107     2            7    1087.49          7
   111     2            7    1155.91          7
   114     2            7    1132.34          7
   116     2            5    1152.05          5
   118     2            7    1108.96          7
    11     0            2    1138.52          2
   120     2            7    1183.65          7
   121     2            0    1138.58          0
   123     2            8    1111.15          8
   128     2            7    1124.17          7
   139     2            3    1168.42          3
    13     0            6    1126.73          6
   141     2            6    1155.44          6
   144     2            3     934.63          3
   150     3            5    1166.40          5
   153     3            5    1115.46        

### MLP with 2 layers

In [8]:
import os
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO


# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = torch.jit.load(r'Models and Data splits\MLP2L.pt')
model.to(device)
model.eval()

# Root directory for your adversarial images
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")

# Gather all PNG files recursively
all_image_paths = list(adversarial_root.rglob("*.png"))

# Dictionary to accumulate results by folder
results_by_folder = {}

for file_path in all_image_paths:
    try:
        # Read as grayscale
        image = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # Normalize and flatten
        image = image.astype(np.float32) / 255.0
        vector = image.flatten().reshape(1, -1)
        input_tensor = torch.tensor(vector, dtype=torch.float32, device=device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_tensor)
        predicted = torch.argmax(outputs, dim=1).item()

        # Determine context folder name (e.g. 'Adversarial' subfolder)
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: '495_t9_p2_m814.39'
        fname = file_path.stem
        parts = fname.split("_")           # ['495', 't9', 'p2', 'm814.39'] :contentReference[oaicite:0]{index=0}

        index      = int(parts[0])
        true_label = int(parts[1].lstrip("t"))  # strip leading 't' :contentReference[oaicite:1]{index=1}
        pred_label = int(parts[2].lstrip("p"))  # strip leading 'p'
        magnitude  = float(parts[3].lstrip("m"))  # strip leading 'm'

        # Append to results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":     index,
            "True":      true_label,
            "Adversarial": pred_label,
            "Magnitude": magnitude,
            "ModelPred": predicted
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Prepare and print/copy output tables
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

# Copy everything to the clipboard
pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")




# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")



=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          2
   103     2            8    1133.78          2
   104     2            7     852.70          8
   107     2            7    1087.49          2
   111     2            7    1155.91          2
   114     2            7    1132.34          2
   116     2            5    1152.05          2
   118     2            7    1108.96          7
    11     0            2    1138.52          0
   120     2            7    1183.65          2
   121     2            0    1138.58          4
   123     2            8    1111.15          2
   128     2            7    1124.17          2
   139     2            3    1168.42          3
    13     0            6    1126.73          0
   141     2            6    1155.44          2
   144     2            3     934.63          2
   150     3            5    1166.40          3
   153     3            5    1115.46        

### CNN

In [10]:
import cv2
import numpy as np
import torch
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO

# 1. Setup device & load your saved CNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(r"Models and Data splits\CNN.pt").to(device)
model.eval()

# 2. Locate all adversarial PNGs
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")
all_image_paths = list(adversarial_root.rglob("*.png"))

results_by_folder = {}

# 3. Inference loop
for file_path in all_image_paths:
    try:
        # Load grayscale image
        img = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # Normalize & reshape to (1,1,H,W)
        img = img.astype(np.float32) / 255.0
        tensor = torch.from_numpy(img).unsqueeze(0).unsqueeze(0).to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(tensor)
        predicted = outputs.argmax(dim=1).item()

        # Determine folder context
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: e.g. "495_t9_p2_m814.39.png"
        fname = file_path.stem
        idx, t_lbl, p_lbl, m_val = fname.split("_")
        index      = int(idx)
        true_label = int(t_lbl.lstrip("t"))
        pred_label = int(p_lbl.lstrip("p"))
        magnitude  = float(m_val.lstrip("m"))

        # Accumulate results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":       index,
            "True":        true_label,
            "Adversarial": pred_label,
            "Magnitude":   magnitude,
            "ModelPred":   predicted
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# 4. Print tables & copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")



# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")



=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          2
   103     2            8    1133.78          2
   104     2            7     852.70          2
   107     2            7    1087.49          2
   111     2            7    1155.91          2
   114     2            7    1132.34          2
   116     2            5    1152.05          2
   118     2            7    1108.96          2
    11     0            2    1138.52          0
   120     2            7    1183.65          2
   121     2            0    1138.58          2
   123     2            8    1111.15          8
   128     2            7    1124.17          2
   139     2            3    1168.42          2
    13     0            6    1126.73          6
   141     2            6    1155.44          2
   144     2            3     934.63          2
   150     3            5    1166.40          3
   153     3            5    1115.46        

### RF

In [12]:
import cv2
import numpy as np
import joblib
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO

# 1. Load your saved Random Forest model
#    Adjust the path/filename to wherever you saved it with joblib.dump(...)
rf_model = joblib.load(r"Models and Data splits\RF.pkl")

# 2. Locate all adversarial PNGs
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")
all_image_paths = list(adversarial_root.rglob("*.png"))

results_by_folder = {}

# 3. Inference loop
for file_path in all_image_paths:
    try:
        # Read as grayscale
        img = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # Normalize pixel values to [0, 1], flatten to 1D feature vector
        img = img.astype(np.float32) / 255.0
        vector = img.flatten().reshape(1, -1)  # shape (1, n_features)

        # Predict with your RF
        pred = rf_model.predict(vector)[0]

        # Determine folder context
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: e.g. "495_t9_p2_m814.39.png"
        fname = file_path.stem
        idx, t_lbl, p_lbl, m_val = fname.split("_")
        index      = int(idx)
        true_label = int(t_lbl.lstrip("t"))
        pred_label = int(p_lbl.lstrip("p"))
        magnitude  = float(m_val.lstrip("m"))

        # Accumulate results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":       index,
            "True":        true_label,
            "Adversarial": pred_label,
            "Magnitude":   magnitude,
            "ModelPred":   int(pred)
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# 4. Prepare tables & copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")



# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")



=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          4
   103     2            8    1133.78          8
   104     2            7     852.70          3
   107     2            7    1087.49          2
   111     2            7    1155.91          2
   114     2            7    1132.34          2
   116     2            5    1152.05          2
   118     2            7    1108.96          2
    11     0            2    1138.52          0
   120     2            7    1183.65          2
   121     2            0    1138.58          2
   123     2            8    1111.15          8
   128     2            7    1124.17          2
   139     2            3    1168.42          3
    13     0            6    1126.73          8
   141     2            6    1155.44          2
   144     2            3     934.63          3
   150     3            5    1166.40          3
   153     3            5    1115.46        

### kNN

In [14]:
import cv2
import numpy as np
import joblib
import pandas as pd
import pyperclip
from pathlib import Path
from io import StringIO

# 1. Load your saved Random Forest model
#    Adjust the path/filename to wherever you saved it with joblib.dump(...)
rf_model = joblib.load(r"Models and Data splits\kNN.pkl")

# 2. Locate all adversarial PNGs
adversarial_root = Path(r"Generated Data\ZC-CSA_images\Adversarial")
all_image_paths = list(adversarial_root.rglob("*.png"))

results_by_folder = {}

# 3. Inference loop
for file_path in all_image_paths:
    try:
        # Read as grayscale
        img = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Could not load image {file_path}")
            continue

        # Normalize pixel values to [0, 1], flatten to 1D feature vector
        img = img.astype(np.float32) / 255.0
        vector = img.flatten().reshape(1, -1)  # shape (1, n_features)

        # Predict with your RF
        pred = rf_model.predict(vector)[0]

        # Determine folder context
        context_folder = file_path.parts[-3] if len(file_path.parts) >= 3 else "UnknownFolder"

        # Parse filename: e.g. "495_t9_p2_m814.39.png"
        fname = file_path.stem
        idx, t_lbl, p_lbl, m_val = fname.split("_")
        index      = int(idx)
        true_label = int(t_lbl.lstrip("t"))
        pred_label = int(p_lbl.lstrip("p"))
        magnitude  = float(m_val.lstrip("m"))

        # Accumulate results
        results_by_folder.setdefault(context_folder, []).append({
            "Index":       index,
            "True":        true_label,
            "Adversarial": pred_label,
            "Magnitude":   magnitude,
            "ModelPred":   int(pred)
        })

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# 4. Prepare tables & copy to clipboard
output = StringIO()
for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    header = f"\n=== Results from Folder: {folder} ===\n"
    table_str = df.to_string(index=False)
    print(header + table_str)
    output.write(header)
    output.write(table_str + "\n")

pyperclip.copy(output.getvalue())
print("\n✅ All formatted tables have been copied to the clipboard!")



# --- Misclassification Summary ---
# Per‐folder
print("\n=== Misclassification Summary by Folder ===")
total_all = 0
mismatch_all = 0

for folder, records in results_by_folder.items():
    df = pd.DataFrame(records)
    total    = len(df)
    mismatches = (df["True"] != df["ModelPred"]).sum()
    pct      = mismatches / total * 100
    print(f"{folder:15s}  {mismatches:4d}/{total:4d} misclassified   ({pct:5.2f}%)")
    total_all    += total
    mismatch_all += mismatches

# Overall
pct_all = mismatch_all / total_all * 100 if total_all else 0.0
print("\nOverall misclassification rate:")
print(f"    {mismatch_all}/{total_all} samples   ({pct_all:.2f}%)")



=== Results from Folder: ZC-CSA_images ===
 Index  True  Adversarial  Magnitude  ModelPred
   102     2            7    1150.38          2
   103     2            8    1133.78          2
   104     2            7     852.70          7
   107     2            7    1087.49          2
   111     2            7    1155.91          2
   114     2            7    1132.34          2
   116     2            5    1152.05          2
   118     2            7    1108.96          2
    11     0            2    1138.52          0
   120     2            7    1183.65          2
   121     2            0    1138.58          2
   123     2            8    1111.15          7
   128     2            7    1124.17          2
   139     2            3    1168.42          2
    13     0            6    1126.73          0
   141     2            6    1155.44          2
   144     2            3     934.63          2
   150     3            5    1166.40          3
   153     3            5    1115.46        